In [24]:
import numpy as np
import pandas as pd
from tqdm import tqdm

## 아이템 상호작용 행렬 만들기

1. 필요한 데이터 로딩

In [2]:
# TODO : 데이터 경로 설정 
raw_data = pd.read_excel("/opt/ml/workspace/crawler/item_crawler_ver2/total_asset/item.xlsx")
df = raw_data[["id", "codi_id"]]
unique_codi_id = list(df["codi_id"].unique())
unique_id = list(df["id"].unique())
codi_group_by_codi = df.groupby("codi_id")["id"].apply(list)
codi_group_by_item = df.groupby("id")["codi_id"].apply(list)

In [84]:
df["id"].value_counts()

1432188    14
2338082    11
2040291     9
1820027     8
996177      8
           ..
2406920     1
2404167     1
2422604     1
2379894     1
2292995     1
Name: id, Length: 2284, dtype: int64

In [82]:
cnt = 0
for count in df["id"].value_counts() : 
    if count == 1 :
        cnt +=1

print(f"Number of total instance : {len(df)}")
print(f"Number of unique items : {len(unique_id)}")
print(f"Number of items which was only collected once : {cnt}")

Number of total instance : 3342
Number of unique items : 2284
Number of items which was only collected once : 1637


2. Pivot Table을 위한 데이터 처리 과정

In [56]:
result = list()
c = 0
for item in tqdm(unique_id) :
    codi_list = codi_group_by_item[item]
    total_data = pd.DataFrame([], dtype=int)

    for codi in codi_list :
        data = df[df["codi_id"]==codi]["id"]
        total_data = pd.concat([total_data, data])
    
    counts = total_data.value_counts()
    id_list = total_data.squeeze().unique()
    
    for id in id_list : 
        if id != item :
            result.append((item, int(id), counts[id]))

100%|██████████| 2284/2284 [00:07<00:00, 301.48it/s]


3. 상호작용 행렬 만들기 & 데이터 내보내기

In [62]:
IIM = pd.DataFrame(result, columns=["id", "item", "count"]).pivot_table(index="id", columns="item", values="count").fillna(0)
# TODO : 데이터 경로 설정
IIM.to_csv("/opt/ml/input/data/itemInteractionMatrix.csv")

4. Sparsity 점검

In [77]:
ones = 0
zeros = 0
for i in range(len(IIM)) :
    zeros += IIM.iloc[i].value_counts()[0]
    ones += IIM.iloc[i].value_counts()[1]
print(f"Sparsity of this Data is : {zeros/(ones+zeros):.4}")

Sparsity of this Data is : 0.997
